In [1]:
class StepLogger():
    def __init__(self,capacity):
        self.tensor_datas = {}        
        self.capacity = capacity
        self.added_labels = []
        
    
    def add_info(self,tensor_data,tensor_label):
        if tensor_label not in self.added_labels:
            self.added_labels.append( tensor_label )
        
        if tensor_label in self.tensor_datas.keys():
            current_arr = self.tensor_datas.get(tensor_label)
            if len(current_arr) < self.capacity:
                current_arr = self.tensor_datas.get(tensor_label, [])
                current_arr.append(tensor_data)
        else:
            self.tensor_datas[tensor_label] = [tensor_data]
    
    def get_default_summary(self,show_data=False,summary_count=1):
        self.get_summary(self.added_labels,show_data)
        
    def get_summary(self,labels,show_data=False,summary_count=1):
        count = 0
        for i in range(summary_count):
            for l in labels:
                label_data = self.tensor_datas.get(l)[count]
                print(l," :")
                if torch.is_tensor(label_data):
                    print( list(label_data.size() ) )
                if not show_data and not torch.is_tensor(label_data):
                    print(label_data)
                if show_data:    
                    print(label_data)

## **N-Grams**

In [4]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["world"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward>)


In [3]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(1)

In [17]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, When forty Summer within thine own deep sunken eyes,
Were an all-eating shame, When forty Summer and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and When forty Summer make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And When forty Summer see thy blood warm when thou feel'st it cold.""".split()


# oluşturalacak liste --> ([ word_i-2, word_i-1 ], target word)

trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 256)
        self.linear2 = nn.Linear(256, vocab_size)

    def forward(self, inputs):

        step_logger.add_info(inputs,"Inputs")
        embeds = self.embeddings(inputs).view((1, -1))

        step_logger.add_info(embeds,"Embeds")
        out = F.relu(self.linear1(embeds))

        step_logger.add_info(out,"Out")
        out = self.linear2(out)

        step_logger.add_info(out,"Out-2")
        log_probs = F.log_softmax(out, dim=1)

        step_logger.add_info(log_probs,"Out-log_probs")
        return log_probs


losses = []
loss_function = nn.NLLLoss()
step_logger =  StepLogger(2)
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

#context = when forty
#target  = winters

        # Step 1. Modele vericek giriş hazırlanıyor..
        
        step_logger.add_info(context,"Giriş Kelimeleri")
        step_logger.add_info(target,"Hedef Kelimeler")
        #print("Context" , context)
        #print("Target : " , target)

        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        #print("context_idxs" , context_idxs)

#context_idxs = [0 , 40]

        # Step 2. Her döngüde gradı sıfırlamak için unutma !!

        model.zero_grad()
        step_logger.add_info(context_idxs,"Input ID")
        # Step 3. Modeli Çalıştırıyoruz
        
        log_probs = model(context_idxs)
        
# verilen iki kelimeden sonra gelebilcek kelimelerin ilişkisini tahmin etmeye çalıştık. döndürdüğü matriste her bir kelime için puan hesapladı. puanı en yüksek kelime ikiliden sonra gelebilcek kelimedir.

        #print("log_probs" , log_probs)

        # Step 4. Loss Fonksiyonu Çağrıldı Ve Loss ne kadar olduğunu gördük.
        
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        step_logger.add_info(log_probs,"Çıkış Boyutu")
# hata miktarımızı gördük
        #print("loss" , loss)

        # Step 5. Geri Yayılım Ve Güncelleme
        loss.backward()
        optimizer.step()

        # Lossu int yaptık.
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

with torch.no_grad():
    context = ['When' ,'forty']# hedef 'abstract'
    context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
    predict = model(context_idxs)
    
    print(ix_to_word[predict.argmax(dim=1).item()])
    print("prdict" , predict)

    print(torch.sort(predict))

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[579.613226890564, 574.3146271705627, 569.0824315547943, 563.9145669937134, 558.8098919391632, 553.7644205093384, 548.7760677337646, 543.842545747757, 538.9630312919617, 534.1353814601898]
Summer
prdict tensor([[-4.5998, -4.9291, -5.0999, -4.6093, -4.1772, -4.8009, -4.4914, -4.6314,
         -4.7853, -4.7526, -4.5573, -4.6274, -4.3923, -4.8827, -4.7588, -4.6507,
         -5.0153, -4.4443, -4.2072, -4.7182, -4.5671, -4.2100, -4.4596, -4.8192,
         -4.2911, -4.8901, -4.5645, -5.1531, -4.6152, -4.4792, -4.7092, -4.7502,
         -5.0860, -4.3482, -4.3498, -4.5491, -4.3841, -4.9810, -4.8751, -4.9382,
         -4.7898, -4.6182, -4.4279, -4.6396, -4.7201, -4.4722, -4.5700, -4.8065,
         -4.8564, -4.9501, -4.6754, -4.9228, -4.4519, -4.6795, -4.8671, -4.7060,
         -4.8989, -4.4131, -4.5746, -4.6581, -4.4561, -4.9229, -4.4557, -5.1387,
         -4.6617, -4.6315, -4.6226, -5.2032, -4.

In [10]:
word_to_ix["Summer"]

71

In [8]:
word_to_ix

{"'This": 48,
 'And': 36,
 'How': 17,
 'If': 16,
 'Proving': 31,
 'Shall': 37,
 'Summer': 71,
 'Then': 26,
 'This': 74,
 'Thy': 97,
 'To': 78,
 'Were': 76,
 'When': 21,
 'Where': 77,
 'Will': 60,
 'a': 19,
 'all': 12,
 'all-eating': 84,
 'an': 38,
 'and': 24,
 'answer': 7,
 'art': 25,
 'asked,': 95,
 'be': 46,
 'beauty': 40,
 "beauty's": 79,
 'being': 59,
 'besiege': 73,
 'blood': 83,
 'brow,': 30,
 'by': 81,
 'child': 72,
 'cold.': 4,
 'couldst': 1,
 'count,': 94,
 'days;': 51,
 'deep': 10,
 "deserv'd": 3,
 'dig': 91,
 "excuse,'": 75,
 'eyes,': 18,
 'fair': 23,
 "feel'st": 45,
 'field,': 11,
 'forty': 69,
 'gazed': 82,
 'held:': 2,
 'his': 86,
 'in': 0,
 'it': 57,
 'lies,': 49,
 'livery': 70,
 'lusty': 63,
 'made': 5,
 'make': 39,
 'mine': 53,
 'more': 13,
 'much': 87,
 'my': 88,
 'new': 50,
 'now,': 90,
 'of': 42,
 'old': 92,
 'old,': 20,
 'on': 89,
 'own': 28,
 'praise': 29,
 'praise.': 68,
 'proud': 6,
 'say,': 61,
 'see': 34,
 'shall': 66,
 'shame,': 56,
 'small': 44,
 'so': 96,
 

## **C-Bow**

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

CONTEXT_SIZE = 2  # 2 sağ 2 sol bakıcaz
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):

        step_logger.add_info(inputs,"Inputs")
        embeds = self.embeddings(inputs)
        
        step_logger.add_info(embeds,"Embeds")
        embeds = torch.sum(embeds, dim=0).view(1,-1)
        step_logger.add_info(embeds,"Embeds")

        
        out = F.relu(self.linear1(embeds))
        step_logger.add_info(out,"Out")

        
        out = self.linear2(out)
        step_logger.add_info(out,"Out - 2")
        
        log_probs = F.log_softmax(out, dim=1)
        step_logger.add_info(log_probs,"Out - log_probs")
        return log_probs


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

losses = []
loss_function = nn.NLLLoss()
model = CBOW(vocab_size, embedding_dim=20)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(200):
    total_loss = 0
    for context, target in data:

        # Step 1. Modele vericek giriş hazırlanıyor..
        step_logger.add_info(context,"Giriş Kelimeleri")
        step_logger.add_info(target,"Hedef Kelimeler")

        context_idxs = make_context_vector(context, word_to_ix)
        step_logger.add_info(context_idxs,"Giriş ID'leri")
        

        
        # Step 2. Her döngüde gradı sıfırlamak için unutma !!

        model.zero_grad()
        
        #  Step 3. Modeli Çalıştırıyoruz

        log_probs = model(context_idxs)
        step_logger.add_info(log_probs,"Çıkış")
        # Loss hesaplanıyor
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Geri Yayılım Ve Güncelleme
        loss.backward()
        optimizer.step()

        
        total_loss += loss.item()

    losses.append(total_loss)

print(losses)  

# Test
with torch.no_grad():
    context = ['manipulate', 'other', 'things', 'called']# hedef 'abstract'
    context_vector = make_context_vector(context, word_to_ix)
    predict = model(context_vector)
    print(ix_to_word[predict.argmax(dim=1).item()])
    

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
[231.63839769363403, 226.3545880317688, 221.23607397079468, 216.2794132232666, 211.46721816062927, 206.7799129486084, 202.222083568573, 197.78033781051636, 193.45168137550354, 189.2250578403473, 185.09620904922485, 181.06700456142426, 177.12904596328735, 173.28111684322357, 169.52237379550934, 165.84369313716888, 162.24597692489624, 158.7208503484726, 155.26600241661072, 151.87890553474426, 148.55104756355286, 145.28077971935272, 142.06333017349243, 138.9022661447525, 135.7928510904312, 132.7371107339859, 129.72949528694153, 126.76932978630066, 123.85904824733734, 121.0003473162651, 118.18883925676346, 115.42624920606613, 112.71280032396317, 110.04614990949631, 107.43204808235168, 104.86308205127716, 102.34725445508957, 99.88433456420898, 97.46948248147964, 95.09938055276871, 92.7818